In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import keras_tuner as kt

from numpy import array
from numpy import hstack
from numpy import empty
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from os import chdir
from os import getcwd
from os import listdir
import math

csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/Sweden_1'
chdir(csv_file_path)

Y_all = pd.read_csv('heads.csv',decimal='.',index_col=0, delimiter=',', header=0,parse_dates=True)
X_all = pd.read_csv('input_data.csv',decimal='.',index_col='time', delimiter=',', header=0,parse_dates=True)

Y = Y_all['2002-06-04':' 2016-12-31']
X = X_all['2002-05-05':' 2016-12-31']
# for sweden weekly forecasts are asked
print(X.index, Y.index)

DatetimeIndex(['2002-05-05', '2002-05-06', '2002-05-07', '2002-05-08',
               '2002-05-09', '2002-05-10', '2002-05-11', '2002-05-12',
               '2002-05-13', '2002-05-14',
               ...
               '2016-12-22', '2016-12-23', '2016-12-24', '2016-12-25',
               '2016-12-26', '2016-12-27', '2016-12-28', '2016-12-29',
               '2016-12-30', '2016-12-31'],
              dtype='datetime64[ns]', name='time', length=5355, freq=None) DatetimeIndex(['2002-06-04', '2002-06-11', '2002-06-18', '2002-06-25',
               '2002-07-02', '2002-07-09', '2002-07-16', '2002-07-23',
               '2002-07-30', '2002-08-06',
               ...
               '2015-10-27', '2015-11-03', '2015-11-10', '2015-11-17',
               '2015-11-24', '2015-12-01', '2015-12-08', '2015-12-15',
               '2015-12-22', '2015-12-29'],
              dtype='datetime64[ns]', name='Date', length=709, freq=None)


In [2]:
X_train = X[:'2011-12-26']
Y_train = Y[:'2011-12-27']
#print(X_train.index, Y_train.index)

X_test = X['2014-01-05':'2014-12-29']
Y_test = Y['2014-02-04':'2014-12-30']#29
#print(X_test.index, Y_test.index)

X_valid = X['2012-01-01':'2013-12-30']
Y_valid = Y['2012-01-31':'2013-12-31']
#print(Y_train.shape, Y_test.shape, Y_valid.shape)

csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\data'
chdir(csv_file_path)
from helper import *

(x_tr,y, x_te, y_test, x_va, y_valid, scaler_X,scaler_Y)=prepare_data_weekly(X_train,Y_train,X_valid,Y_valid,X_test,Y_test)

iters=3
n_steps_in=30
n_steps_out=1

x = split_sequences_x(x_tr, n_steps_in, n_steps_out)
x_test = split_sequences_x(x_te, n_steps_in, n_steps_out)
x_valid = split_sequences_x(x_va, n_steps_in, n_steps_out)
print(x_test.shape, x.shape, x_valid.shape)
print(y_test.shape, y.shape, y_valid.shape)

DatetimeIndex(['2002-05-05', '2002-05-06', '2002-05-07', '2002-05-08',
               '2002-05-09', '2002-05-10', '2002-05-11', '2002-05-12',
               '2002-05-13', '2002-05-14',
               ...
               '2011-12-17', '2011-12-18', '2011-12-19', '2011-12-20',
               '2011-12-21', '2011-12-22', '2011-12-23', '2011-12-24',
               '2011-12-25', '2011-12-26'],
              dtype='datetime64[ns]', name='time', length=3523, freq=None) DatetimeIndex(['2002-06-04', '2002-06-11', '2002-06-18', '2002-06-25',
               '2002-07-02', '2002-07-09', '2002-07-16', '2002-07-23',
               '2002-07-30', '2002-08-06',
               ...
               '2011-10-25', '2011-11-01', '2011-11-08', '2011-11-15',
               '2011-11-22', '2011-11-29', '2011-12-06', '2011-12-13',
               '2011-12-20', '2011-12-27'],
              dtype='datetime64[ns]', name='Date', length=500, freq=None)
DatetimeIndex(['2014-01-05', '2014-01-06', '2014-01-07', '2014-01-08',


In [4]:
from keras_tuner import HyperModel
from tensorflow.keras import layers

class Transformer_HyperModel(HyperModel):
    def __init__(self, input_shape, output_shape):
        super(Transformer_HyperModel, self).__init__()
        
        self.input_shape = input_shape
        self.output_shape = output_shape
        
    def build(self, hp):#statt build_model
        head_size = hp.Int("head_size", min_value=8, max_value=64) #, default=16
        num_heads = 1
        ff_dim = hp.Int("ff_dim", min_value=1, max_value=8) #, default=4
        num_transformer_blocks = 2 #, default=4
        mlp_units = hp.Int("mlp_units", min_value=10, max_value=200)
        dropout = 0.1
        mlp_dropout = 0.1
        
        model = build_model(self.input_shape, self.output_shape, head_size,num_heads,ff_dim,
                                num_transformer_blocks,mlp_units,mlp_dropout)
        model.compile(loss="mean_absolute_error",
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),metrics=["mean_absolute_error"])
        return model

input_shape = x.shape[1:]
output_shape = y.shape[1]

hyper_model = Transformer_HyperModel(input_shape, output_shape)

es_callback = keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error',restore_best_weights=True, patience=4)




In [5]:
tuner= kt.RandomSearch(
        hyper_model,
        overwrite=True,
        objective='val_mean_absolute_error',
        max_trials = 10,
        directory='D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge',
        project_name='TF Sweden1'
        )

tuner.search(
        x,
        y,
        batch_size=24,
        epochs=30,
        validation_data=(x_valid,y_valid),#achtung test-set hier darf nicht mein standard test set sein, sondern val-set!! 
        callbacks=[es_callback],
        verbose=1)

best_model = tuner.get_best_models(num_models=1)[0]

Y_pred=best_model.predict(x_test)
best_model.summary()

Trial 10 Complete [00h 00m 06s]
val_mean_absolute_error: 0.1269529014825821

Best val_mean_absolute_error So Far: 0.11397659033536911
Total elapsed time: 00h 01m 33s
INFO:tensorflow:Oracle triggered exit
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 9)]      0                                            
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 30, 9)        18          input_1[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 30, 9)        1803        layer_normalization[0][0]        
                                                                 layer_normalization[0][

In [6]:
tuner.results_summary()

csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge'
chdir(csv_file_path)
from tensorflow.keras.models import save_model
best_model.save('TF_Sweden1_tune.h5')

NSE=np.array([[None]*n_steps_out])
MAE=np.array([[None]*n_steps_out])
RMSE=np.array([[None]*n_steps_out])
SAPE=np.array([[None]*n_steps_out])

Y_pred_test=best_model.predict(x_test)
Y_pred_valid=best_model.predict(x_valid)

y_correct_utf = scaler_Y.inverse_transform(y_test)#test
yhat_utf = scaler_Y.inverse_transform(Y_pred_test)#test
Y_pred_valid_utf = scaler_Y.inverse_transform(Y_pred_valid)#valid

NSE[:]= np.array([1 - sum((yhat_utf[:,i]-y_correct_utf[:,i])**2)/sum((y_correct_utf[:,i]-np.mean(y_correct_utf[:,i]))**2)
                  for i in range(n_steps_out)])
MAE[:]=np.array([mean_absolute_error(y_correct_utf[:,i], yhat_utf[:,i]) for i in range(n_steps_out)])
RMSE[:]=np.array([np.sqrt(np.mean((yhat_utf[:,i]-y_correct_utf[:,i])**2)) for i in range(n_steps_out)])
SAPE[:] = np.array([np.abs((yhat_utf[:,i] - y_correct_utf[:,i])/(0.5*(y_correct_utf[:,i]+yhat_utf[:,i]))).mean()
                         for i in range(n_steps_out)]) #mean absolute percent error

print(np.mean(MAE))

Results summary
Results in D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\TF Sweden1
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
head_size: 46
ff_dim: 6
mlp_units: 172
Score: 0.11397659033536911
Trial summary
Hyperparameters:
head_size: 22
ff_dim: 4
mlp_units: 117
Score: 0.11455947905778885
Trial summary
Hyperparameters:
head_size: 53
ff_dim: 7
mlp_units: 189
Score: 0.11616384983062744
Trial summary
Hyperparameters:
head_size: 27
ff_dim: 6
mlp_units: 105
Score: 0.1167249009013176
Trial summary
Hyperparameters:
head_size: 43
ff_dim: 3
mlp_units: 36
Score: 0.11826717853546143
Trial summary
Hyperparameters:
head_size: 53
ff_dim: 6
mlp_units: 51
Score: 0.11877071857452393
Trial summary
Hyperparameters:
head_size: 8
ff_dim: 5
mlp_units: 99
Score: 0.1269529014825821
Trial summary
Hyperparameters:
head_size: 38
ff_dim: 4
mlp_units: 57
Score: 0.1357397586107254
Trial summary
Hyperparameters:
head_size: 38
ff_dim: 6
